# Sprint　深層学習スクラッチ　リカレントニューラルネットワーク

### ＜目的＞
スクラッチを通してリカレントニューラルネットワークの基礎を理解する

リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

### 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)
$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)


$x_{t}$ : 時刻tの入力 (batch_size, n_features)


$W_{x}$ : 入力に対する重み (n_features, n_nodes)


$h_{t-1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


$W_{h}$ : 状態に対する重み。 (n_nodes, n_nodes)


$B$ : バイアス項 (n_nodes,)


初期状態 $h_{0}$ は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。



In [15]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import copy
from sklearn import metrics 
from scipy.special import expit

### 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [16]:
# 例の配列
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

In [ ]:
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) 
# (batch_size, n_nodes)

In [17]:
# forward関数
def forward(X, h):
    A = X @ w_x + h @ w_h + b
    return np.tanh(A)

In [18]:
# シーケンス数を取得し、xを変形
n_sequence = x.shape[1]
x = x.transpose(1,0,2)

# シーケンス数だけループを回す
# 時刻（シーケンス）t のXでhを更新していく
for i in range(n_sequence):
    h = forward(x[i], h)
    
print('h :',h)

h : [[0.79494228 0.81839002 0.83939649 0.85584174]]


正しい出力になっているようだ。

### 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$
W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\
W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\
B^{\prime} = B - \alpha \frac{\partial L}{\partial B}
$$

$\alpha$ : 学習率


$\frac{\partial L}{\partial W_x}$ : $W_x$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial W_h}$ : $W_h$ に関する損失 $L$ の勾配


$\frac{\partial L}{\partial B}$ : $B$ に関する損失 $L$ の勾配


勾配を求めるためのバックプロパゲーションの数式が以下です。


$\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} ×(1-tanh^2(a_t))$


$\frac{\partial L}{\partial B} = \frac{\partial h_t}{\partial a_t}$


$\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}$


$\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}$


＊$\frac{\partial L}{\partial h_t}$ は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。


前の時刻や層に流す誤差の数式は以下です。


$\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}$


$\frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}$

In [19]:
class SimpleRNN:
    """
    RNNの全結合層
    Parameters
    ----------
    n_features : int
      入力の特徴量数
    n_nodes : int
      層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_features, n_nodes, batch_size, initializer, activator, optimizer):
        
        # 各パラメーターの初期化
        self.batch_size = batch_size
        self.n_nodes = n_nodes
        self.Wx = initializer.param(n_features, n_nodes)
        self.Wh = initializer.param(n_nodes, n_nodes)
        self.B = initializer.param(n_nodes, 1).flatten()
        # backward時のパラメーター初期化
        self.X = []
        self.h = []
        self.dWx = 0
        self.dWh = 0
        self.dB = 0
        # 活性化関数、最適化手法
        self.activator = activator
        self.optimizer = optimizer
    
    # 順伝播
    def forward(self, X, h):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_features)
            入力
        h : 次の形のndarray, shape (batch_size, n_nodes)
            前の時刻の状態・入力
        Returns
        ----------
        h : 次の形のndarray, shape (batch_size, n_features)
            出力
        """
        # 時刻 t における変数をリストで保持
        self.X.append(X)
        self.h.append(h)
        
        # Affineと同様の計算
        A = X @ self.Wx + h @ self.Wh + self.B
        
        return self.activator.activate(A)
    
    # 逆伝播
    def backward(self, dX, dh, t):
        """
        バックワード
        Parameters
        ----------
        dX : 次の形のndarray, shape (batch_size, n_features)
            後ろから流れてきた勾配
        dh : 次の形のndarray, shape (batch_size, n_nodes)
            後ろから流れてきた勾配
        t  : int
            時刻
        Returns
        ----------
        dX : 次の形のndarray, shape (batch_size, n_features)
            前に流す勾配
        dh : 次の形のndarray, shape (batch_size, n_nodes)
            前に流す勾配
        """
        # 活性化関数のヤコビアンを通し、dh/da(dA)を得る
        dA = self.activator.jacobian(dh)
        
        # 時刻 t における変数を取り出す
        X = self.X[t]
        h = self.h[t]
        
        # X,hとバッチサイズでE(dWx), E(dWh)を算出
        self.dWx += (X.T @ dA) / self.batch_size
        self.dWh += (h.T @ dA) / self.batch_size
        self.dB += np.sum(dA, axis=0)
        
        # 前層に流す勾配を計算
        dX = dA @ (self.Wx).T
        dh = dA @ (self.Wh).T
        
        # t=0 になったらパラメーター更新
        if t==0:
            self.optimizer.update(self)
        
        return dX, dh

In [20]:
# ReccurentNeuralNetクラス
class ScratchSipmleRNNClassifier():
    
    def __init__(self, n_nodes, sigma, lr):
        self.n_nodes = n_nodes
        self.sigma = sigma
        self.lr = lr
        
    def fit(self, X, h, y):
        # 入力データからシーケンス数等を取得し変形
        batch_size, n_sequences, n_features = X.shape
        X = X.transpose(1,0,2)
        
        # RNN層のインスタンス作成（単層）
        self.rnn = SimpleRNN(n_features=n_features, 
                             n_nodes=self.n_nodes, 
                             batch_size=batch_size,
                             initializer=SimpleInitializer(self.sigma), 
                             activator=Tanh(), 
                             optimizer=SGD(self.lr))
        
        # シーケンス数だけforwardし、dhのみ更新
        for t in range(n_sequence):
            h = self.rnn.forward(X[t], h)
            print('\nforward(t = {})'.format(t))
            print('h = ', h)
        
        # ここはタスクによって変わるが、ひとまずは単純な差とした
        dh = h - y
        
        # backward前に、dXを空の配列で初期化
        dX = np.empty([batch_size, n_features])
        
        # シーケンス数だけbackwaordし、dX, dhを更新
        # 時刻を引数に与え、t=0の時にパラメーター更新
        for t in reversed(range(n_sequence)):
            dX, dh = self.rnn.backward(dX, dh, t)
            print('\nbackward(t = {})'.format(t))
            print('dh = ', dh)
            print('dX = ', dX)
        
        return self

    
    def predict(self, X, h):
        # 入力データからシーケンス数等を取得し変形
        batch_size, n_sequences, n_features = X.shape
        X = X.transpose(1,0,2)
        # シーケンス数だけforward
        for i in range(n_sequence):
            h = self.rnn.forward(X[i], h)
        # ここでは出力をそのまま出すものとした
        return h

In [21]:
# ハイパボリックタンジェント関数
class Tanh:
    # 順伝播時の関数メソッド（softmaxに合わせ、yも引数にとるが使用しない）
    def activate(self, A):
        # 逆伝播時に使用する変数を保持
        self.Z = np.tanh(A)
        return self.Z
    
    # 逆伝播時のヤコビアン計算メソッド
    def jacobian(self, dZ):
        dA = dZ*(1. - self.Z**2)
        return dA

In [22]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def param(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1, n_nodes_2 : int
          パラメーターサイズ
        
        Returns
        ----------
        param : 次の形のndarray, shape (n_nodes1, n_nodes2)
          初期化されたパラメーター
        """
        param = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return param

In [23]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    
    def update(self, layer):
        """
        ある結合層のパラメーターの更新
        Parameters
        ----------
        layer : object
          更新前の層のインスタンス
        """
        layer.Wx -= self.lr*layer.dWx
        layer.Wh -= self.lr*layer.dWh
        layer.B -= self.lr*layer.dB
        return self

In [24]:
# サンプル配列で確認
X = np.array([[[1, 2], [2, 3], [3, 4]]])/100
h = np.zeros((1, 4)) 
y = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

# 学習
srnn = ScratchSipmleRNNClassifier(n_nodes=4, 
                                  sigma=0.1, 
                                  lr=0.01)
srnn.fit(X, h, y)

# 推論
h = np.zeros((1, 4)) 
pred = srnn.predict(X, h)
print('\n\npredict = ', pred)


forward(t = 0)
h =  [[ 0.15319695  0.05211778 -0.05504374  0.01341288]]

forward(t = 1)
h =  [[ 0.14033102  0.04131364 -0.06229699  0.01548842]]

forward(t = 2)
h =  [[ 0.1437431   0.04180554 -0.05926674  0.01241166]]

backward(t = 2)
dh =  [[ 0.06236581  0.18715963 -0.06993082 -0.10554729]]
dX =  [[-0.05391941  0.03441599]]

backward(t = 1)
dh =  [[-0.00909855 -0.02229231  0.00891267  0.03386782]]
dX =  [[ 0.03840553 -0.03300507]]

backward(t = 0)
dh =  [[ 0.00263497  0.0019623   0.00051703 -0.00470682]]
dX =  [[-0.00732095  0.00226414]]


predict =  [[ 0.1500549   0.04876637 -0.0509964   0.02242665]]


少なくとも配列の形状は合っているようだ